In [ ]:
# First, uninstall the existing PyTorch installation
!pip uninstall torch torchvision torchaudio -y

!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

print("PyTorch imported successfully!")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import seaborn as sns
import random
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import grangercausalitytests
from sklearn.linear_model import LinearRegression
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/CPCB.csv", parse_dates=['Datetime'], dayfirst=True)
# data = pd.read_csv('/content/drive/MyDrive/CPCB.csv', parse_dates=['Datetime'])

In [ ]:
data.shape

In [ ]:
data.describe()

In [ ]:
data.info()

In [ ]:
data.isnull().sum()

In [ ]:
df = pd.DataFrame(data)

def replace_outliers_with_median(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    median = df[column].median()

    # Replace outliers with median
    df[column] = df[column].apply(lambda x: median if x < lower_bound or x > upper_bound else x)
    return df

# Apply to the column
df_cleaned = replace_outliers_with_median(df, 'PM2.5')
print(df_cleaned)

In [ ]:
df = pd.DataFrame(df_cleaned)

# Check for duplicate rows
duplicate_mask = df.duplicated()

# Print duplicate rows
print("Duplicate rows:\n", df[duplicate_mask])

df_no_duplicates = df.drop_duplicates()

Stationarity check using ADF

In [ ]:
def check_stationarity_adf(ata):
    results = {}
    for col in df.select_dtypes(include='number').columns:
        series = data[col].dropna()
        adf_result = adfuller(series)

        results[col] = {
            'ADF Statistic': adf_result[0],
            'p-value': adf_result[1],
            'Stationary': adf_result[1] < 0.05  # True if p < 0.05
        }

    return pd.DataFrame(results).T

df = pd.DataFrame(data)

adf_results = check_stationarity_adf(df)
print(adf_results)

Correlation coefficient check

In [ ]:
# Compute correlation matrix
corr_matrix = df.select_dtypes(include=np.number).corr()

# Plot heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap='coolwarm', square=True, linewidths=0.5)
plt.title("Correlation Heatmap")
plt.show()

Granger Test

In [ ]:
# Suppose 'data' is your scaled or differenced dataframe with columns: ['PM2.5', 'SO2', 'CO', 'PM10']
df_numeric = df.select_dtypes(include=['float64', 'int64'])

# 🔹 2. Granger Causality Tests

# Choose the maximum lag based on your VAR model selection (example: 5)
maxlag = 5

print("\nGranger Causality Tests:")
for target in df_numeric.columns:
    print(f"\nTarget Variable: {target}")
    for predictor in df_numeric.columns:
        if target != predictor:
            print(f"  Testing if {predictor} causes {target}")
            test_result = grangercausalitytests(df_numeric[[target, predictor]], maxlag=maxlag, verbose=False)
            # Print p-values for each lag
            for lag in range(1, maxlag + 1):
                p_value = test_result[lag][0]['ssr_ftest'][1]
                print(f"    Lag {lag}: p-value = {p_value:.4f}")

In [ ]:
# PM2.5
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.linear_model import Ridge

# Load and prepare data
df = pd.read_csv('/content/drive/MyDrive/CPCB.csv', parse_dates=['Datetime'])
df.set_index('Datetime', inplace=True)

features = ['PM2.5', 'SO2', 'CO', 'PM10', 'Temp', 'RH']
df['PM2.5_roll3'] = df['PM2.5'].rolling(9).mean()
df['Temp_roll3'] = df['Temp'].rolling(9).mean()
df = df.dropna()

df = df[features].dropna()

# Split data
n = len(df)
train_end = int(n * 0.7)
val_end = int(n * 0.85)

train = df.iloc[:train_end]
val = df.iloc[train_end:val_end]
test = df.iloc[val_end:]

# Lag function
def create_lagged_data(df, lags=1):
    df_lagged = df.copy()
    for col in df.columns:
        for lag in range(1, lags + 1):
            df_lagged[f'{col}_lag{lag}'] = df[col].shift(lag)
    return df_lagged.dropna()

# TAR model with polynomial features
def fit_tar_model_poly(df_lagged, threshold):
    poly = PolynomialFeatures(degree=1, include_bias=False)

    regime_low = df_lagged[df_lagged['PM2.5_lag1'] <= threshold]
    regime_high = df_lagged[df_lagged['PM2.5_lag1'] > threshold]

    X_low = regime_low.drop(columns='PM2.5')
    y_low = regime_low['PM2.5']
    X_high = regime_high.drop(columns='PM2.5')
    y_high = regime_high['PM2.5']

    X_low_poly = poly.fit_transform(X_low)
    X_high_poly = poly.fit_transform(X_high)

    model_low = Ridge(alpha=0.1).fit(X_low_poly, y_low)
    model_high = Ridge(alpha=0.1).fit(X_high_poly, y_high)


    return model_low, model_high, threshold, poly

# Prediction function
def predict_tar_poly(df_lagged, model_low, model_high, threshold, poly):
    X = df_lagged.drop(columns='PM2.5')
    X_poly = poly.transform(X)
    condition = df_lagged['PM2.5_lag1'] <= threshold
    preds = np.where(condition,
                     model_low.predict(X_poly),
                     model_high.predict(X_poly))
    return preds

# Grid search over threshold
df_lagged = create_lagged_data(df, lags=1)

train_lagged = df_lagged.loc[train.index.intersection(df_lagged.index)]
val_lagged = df_lagged.loc[val.index.intersection(df_lagged.index)]
test_lagged = df_lagged.loc[test.index.intersection(df_lagged.index)]

# Manually set best threshold and fit model
best_thresh = 10.83
model_low, model_high, _, poly = fit_tar_model_poly(train_lagged, best_thresh)
best_models = (model_low, model_high, poly)

# Optional: evaluate val R² again
yhat_val = predict_tar_poly(val_lagged, model_low, model_high, best_thresh, poly)
best_r2 = r2_score(val_lagged['PM2.5'], yhat_val)
print(f"\n Best threshold used: {best_thresh:.2f} with validation R²: {best_r2:.4f}")


# Final model
model_low, model_high, poly = best_models

# Final predictions
y_train = train_lagged['PM2.5']
y_val = val_lagged['PM2.5']
y_test = test_lagged['PM2.5']

yhat_train = predict_tar_poly(train_lagged, model_low, model_high, best_thresh, poly)
yhat_val = predict_tar_poly(val_lagged, model_low, model_high, best_thresh, poly)
yhat_test = predict_tar_poly(test_lagged, model_low, model_high, best_thresh, poly)

# Metrics
def mase(y_true, y_pred, naive_pred):
    return np.mean(np.abs(y_true - y_pred)) / np.mean(np.abs(y_true[1:] - naive_pred[:-1]))

def evaluate(y_true, y_pred):
    naive = y_true.shift(1).dropna()
    y_true = y_true.iloc[1:]
    y_pred = y_pred[1:]
    return {
        'R2': r2_score(y_true, y_pred),
        'MAE': mean_absolute_error(y_true, y_pred),
        'RMSE': np.sqrt(mean_squared_error(y_true, y_pred)),
        'MASE': mase(y_true, y_pred, naive)
    }

print("\n Final Evaluation with Polynomial TAR:")
print("Train:", evaluate(y_train, yhat_train))
print("Val:  ", evaluate(y_val, yhat_val))
print("Test: ", evaluate(y_test, yhat_test))

# Plots
def plot_actual_vs_pred(y_true, y_pred, title):
    plt.figure(figsize=(10, 4))
    plt.plot(y_true.index, y_true, label='Actual')
    plt.plot(y_true.index, y_pred, label='Predicted')
    plt.title(title)
    plt.xlabel('Time')
    plt.ylabel('PM2.5')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

plot_actual_vs_pred(y_train, yhat_train, 'Train: Actual vs Predicted')
plot_actual_vs_pred(y_val, yhat_val, 'Validation: Actual vs Predicted')
plot_actual_vs_pred(y_test, yhat_test, 'Test: Actual vs Predicted')

# Residuals ACF and PACF
residuals = y_test - yhat_test
plt.figure(figsize=(10, 4))
plot_acf(residuals, lags=30)
plt.title("ACF of Test Residuals")
plt.show()

plt.figure(figsize=(10, 4))
plot_pacf(residuals, lags=30)
plt.title("PACF of Test Residuals")
plt.show()

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import ParameterGrid
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

# Create lagged features for residual correction (train only on past data)
def create_residual_features(df_lagged, residuals, lags=1):
    df_feat = df_lagged.copy()
    df_feat['residuals'] = residuals
    for lag in range(1, lags + 1):
        df_feat[f'residuals_lag{lag}'] = df_feat['residuals'].shift(lag)
    df_feat = df_feat.dropna()
    return df_feat

# Step 1: Compute residuals for train/val/test
residuals_train = y_train - yhat_train
residuals_val = y_val - yhat_val
residuals_test = y_test - yhat_test

# Step 2: Add residuals to original lagged features
train_feat = create_residual_features(train_lagged.copy(), residuals_train, lags=1)
val_feat = create_residual_features(val_lagged.copy(), residuals_val, lags=1)
test_feat = create_residual_features(test_lagged.copy(), residuals_test, lags=1)

# Ensure alignment
res_target_train = train_feat['residuals']
res_target_val = val_feat['residuals']
res_target_test = test_feat['residuals']

# Drop target from features
train_feat_X = train_feat.drop(columns=['residuals'])
val_feat_X = val_feat.drop(columns=['residuals'])
test_feat_X = test_feat.drop(columns=['residuals'])
# max_depth': 7, 'max_features': 'sqrt', 'min_samples_leaf': 5, 'n_estimators': 100

# Step 3: Fit Random Forest on training residuals
rf = RandomForestRegressor(n_estimators=30, random_state=42)
rf.fit(train_feat_X, res_target_train)

# Step 4: Predict residuals and add to TAR prediction
res_pred_train = rf.predict(train_feat_X)
res_pred_val = rf.predict(val_feat_X)
res_pred_test = rf.predict(test_feat_X)

# Align predicted residuals with TAR predictions
yhat_train_corr = yhat_train[-len(res_pred_train):] + res_pred_train
yhat_val_corr = yhat_val[-len(res_pred_val):] + res_pred_val
yhat_test_corr = yhat_test[-len(res_pred_test):] + res_pred_test

# True values (aligned)
y_train_corr = y_train[-len(res_pred_train):]
y_val_corr = y_val[-len(res_pred_val):]
y_test_corr = y_test[-len(res_pred_test):]

# Step 5: Evaluate corrected predictions
print("\n Final Evaluation after Residual Correction with RF:")
print("Train:", evaluate(y_train_corr, yhat_train_corr))
print("Val:  ", evaluate(y_val_corr, yhat_val_corr))
print("Test: ", evaluate(y_test_corr, yhat_test_corr))

# Plot corrected vs actual
plot_actual_vs_pred(y_test_corr, yhat_test_corr, "Test: Actual vs Corrected TAR+RF Forecast")

# ACF/PACF of corrected residuals
final_residuals = y_test_corr - yhat_test_corr
plt.figure(figsize=(10, 4))
plot_acf(final_residuals, lags=30)
plt.title("ACF of Final Residuals (TAR + RF)")
plt.show()

plt.figure(figsize=(10, 4))
plot_pacf(final_residuals, lags=30)
plt.title("PACF of Final Residuals (TAR + RF)")
plt.show()

# Final Evaluation Metrics for Corrected Forecast

print("\n Final Evaluation Metrics (TAR + RF Residual Correction):")
metrics_train = evaluate(y_train_corr, yhat_train_corr)
metrics_val = evaluate(y_val_corr, yhat_val_corr)
metrics_test = evaluate(y_test_corr, yhat_test_corr)

# Print all metrics cleanly
def print_metrics(metrics, label):
    print(f"\n {label} Set Metrics:")
    for k, v in metrics.items():
        print(f"{k}: {v:.4f}")

print_metrics(metrics_train, "Train")
print_metrics(metrics_val, "Validation")
print_metrics(metrics_test, "Test")

In [ ]:
# PM10
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.linear_model import Ridge

# Load and prepare data
df = pd.read_csv('/content/drive/MyDrive/CPCB.csv', parse_dates=['Datetime'])
df.set_index('Datetime', inplace=True)

features = ['PM2.5', 'SO2', 'CO', 'PM10', 'Temp', 'RH']
df['PM10_roll3'] = df['PM10'].rolling(9).mean()
df['Temp_roll3'] = df['Temp'].rolling(9).mean()
df = df.dropna()

df = df[features].dropna()

# Split data
n = len(df)
train_end = int(n * 0.7)
val_end = int(n * 0.85)

train = df.iloc[:train_end]
val = df.iloc[train_end:val_end]
test = df.iloc[val_end:]

# Lag function
def create_lagged_data(df, lags=1):
    df_lagged = df.copy()
    for col in df.columns:
        for lag in range(1, lags + 1):
            df_lagged[f'{col}_lag{lag}'] = df[col].shift(lag)
    return df_lagged.dropna()

# TAR model with polynomial features
def fit_tar_model_poly(df_lagged, threshold):
    poly = PolynomialFeatures(degree=1, include_bias=False)

    regime_low = df_lagged[df_lagged['PM10_lag1'] <= threshold]
    regime_high = df_lagged[df_lagged['PM10_lag1'] > threshold]

    X_low = regime_low.drop(columns='PM10')
    y_low = regime_low['PM10']
    X_high = regime_high.drop(columns='PM10')
    y_high = regime_high['PM10']

    X_low_poly = poly.fit_transform(X_low)
    X_high_poly = poly.fit_transform(X_high)

    model_low = Ridge(alpha=0.01).fit(X_low_poly, y_low)
    model_high = Ridge(alpha=0.01).fit(X_high_poly, y_high)


    return model_low, model_high, threshold, poly

# Prediction function
def predict_tar_poly(df_lagged, model_low, model_high, threshold, poly):
    X = df_lagged.drop(columns='PM10')
    X_poly = poly.transform(X)
    condition = df_lagged['PM10_lag1'] <= threshold
    preds = np.where(condition,
                     model_low.predict(X_poly),
                     model_high.predict(X_poly))
    return preds

# Grid search over threshold
df_lagged = create_lagged_data(df, lags=1)

train_lagged = df_lagged.loc[train.index.intersection(df_lagged.index)]
val_lagged = df_lagged.loc[val.index.intersection(df_lagged.index)]
test_lagged = df_lagged.loc[test.index.intersection(df_lagged.index)]

# Manually set best threshold and fit model
best_thresh = 16.94
model_low, model_high, _, poly = fit_tar_model_poly(train_lagged, best_thresh)
best_models = (model_low, model_high, poly)

# Optional: evaluate val R² again
yhat_val = predict_tar_poly(val_lagged, model_low, model_high, best_thresh, poly)
best_r2 = r2_score(val_lagged['PM10'], yhat_val)
print(f"\n Best threshold used: {best_thresh:.2f} with validation R²: {best_r2:.4f}")


# Final model
model_low, model_high, poly = best_models

# Final predictions
y_train = train_lagged['PM10']
y_val = val_lagged['PM10']
y_test = test_lagged['PM10']

yhat_train = predict_tar_poly(train_lagged, model_low, model_high, best_thresh, poly)
yhat_val = predict_tar_poly(val_lagged, model_low, model_high, best_thresh, poly)
yhat_test = predict_tar_poly(test_lagged, model_low, model_high, best_thresh, poly)

# Metrics
def mase(y_true, y_pred, naive_pred):
    return np.mean(np.abs(y_true - y_pred)) / np.mean(np.abs(y_true[1:] - naive_pred[:-1]))

def evaluate(y_true, y_pred):
    naive = y_true.shift(1).dropna()
    y_true = y_true.iloc[1:]
    y_pred = y_pred[1:]
    return {
        'R2': r2_score(y_true, y_pred),
        'MAE': mean_absolute_error(y_true, y_pred),
        'RMSE': np.sqrt(mean_squared_error(y_true, y_pred)),
        'MASE': mase(y_true, y_pred, naive)
    }

print("\n Final Evaluation with Polynomial TAR:")
print("Train:", evaluate(y_train, yhat_train))
print("Val:  ", evaluate(y_val, yhat_val))
print("Test: ", evaluate(y_test, yhat_test))

# Plots
def plot_actual_vs_pred(y_true, y_pred, title):
    plt.figure(figsize=(10, 4))
    plt.plot(y_true.index, y_true, label='Actual')
    plt.plot(y_true.index, y_pred, label='Predicted')
    plt.title(title)
    plt.xlabel('Time')
    plt.ylabel('PM10')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

plot_actual_vs_pred(y_train, yhat_train, 'Train: Actual vs Predicted')
plot_actual_vs_pred(y_val, yhat_val, 'Validation: Actual vs Predicted')
plot_actual_vs_pred(y_test, yhat_test, 'Test: Actual vs Predicted')

# Residuals ACF and PACF
residuals = y_test - yhat_test
plt.figure(figsize=(10, 4))
plot_acf(residuals, lags=30)
plt.title("ACF of Test Residuals")
plt.show()

plt.figure(figsize=(10, 4))
plot_pacf(residuals, lags=30)
plt.title("PACF of Test Residuals")
plt.show()

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import ParameterGrid
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score


# Create lagged features for residual correction (train only on past data)
def create_residual_features(df_lagged, residuals, lags=1):
    df_feat = df_lagged.copy()
    df_feat['residuals'] = residuals
    for lag in range(1, lags + 1):
        df_feat[f'residuals_lag{lag}'] = df_feat['residuals'].shift(lag)
    df_feat = df_feat.dropna()
    return df_feat

# Step 1: Compute residuals for train/val/test
residuals_train = y_train - yhat_train
residuals_val = y_val - yhat_val
residuals_test = y_test - yhat_test

# Step 2: Add residuals to original lagged features
train_feat = create_residual_features(train_lagged.copy(), residuals_train, lags=1)
val_feat = create_residual_features(val_lagged.copy(), residuals_val, lags=1)
test_feat = create_residual_features(test_lagged.copy(), residuals_test, lags=1)

# Ensure alignment
res_target_train = train_feat['residuals']
res_target_val = val_feat['residuals']
res_target_test = test_feat['residuals']

# Drop target from features
train_feat_X = train_feat.drop(columns=['residuals'])
val_feat_X = val_feat.drop(columns=['residuals'])
test_feat_X = test_feat.drop(columns=['residuals'])
# max_depth': 7, 'max_features': 'sqrt', 'min_samples_leaf': 5, 'n_estimators': 100

# Step 3: Fit Random Forest on training residuals
rf = RandomForestRegressor(n_estimators=30, random_state=42)
rf.fit(train_feat_X, res_target_train)

# Step 4: Predict residuals and add to TAR prediction
res_pred_train = rf.predict(train_feat_X)
res_pred_val = rf.predict(val_feat_X)
res_pred_test = rf.predict(test_feat_X)

# Align predicted residuals with TAR predictions
yhat_train_corr = yhat_train[-len(res_pred_train):] + res_pred_train
yhat_val_corr = yhat_val[-len(res_pred_val):] + res_pred_val
yhat_test_corr = yhat_test[-len(res_pred_test):] + res_pred_test

# True values (aligned)
y_train_corr = y_train[-len(res_pred_train):]
y_val_corr = y_val[-len(res_pred_val):]
y_test_corr = y_test[-len(res_pred_test):]

# Step 5: Evaluate corrected predictions
print("\n Final Evaluation after Residual Correction with RF:")
print("Train:", evaluate(y_train_corr, yhat_train_corr))
print("Val:  ", evaluate(y_val_corr, yhat_val_corr))
print("Test: ", evaluate(y_test_corr, yhat_test_corr))

# Plot corrected vs actual
plot_actual_vs_pred(y_test_corr, yhat_test_corr, "Test: Actual vs Corrected TAR+RF Forecast")

# ACF/PACF of corrected residuals
final_residuals = y_test_corr - yhat_test_corr
plt.figure(figsize=(10, 4))
plot_acf(final_residuals, lags=30)
plt.title("ACF of Final Residuals (TAR + RF)")
plt.show()

plt.figure(figsize=(10, 4))
plot_pacf(final_residuals, lags=30)
plt.title("PACF of Final Residuals (TAR + RF)")
plt.show()

# Final Evaluation Metrics for Corrected Forecast

print("\n Final Evaluation Metrics (TAR + RF Residual Correction):")
metrics_train = evaluate(y_train_corr, yhat_train_corr)
metrics_val = evaluate(y_val_corr, yhat_val_corr)
metrics_test = evaluate(y_test_corr, yhat_test_corr)

# Print all metrics cleanly
def print_metrics(metrics, label):
    print(f"\n {label} Set Metrics:")
    for k, v in metrics.items():
        print(f"{k}: {v:.4f}")

print_metrics(metrics_train, "Train")
print_metrics(metrics_val, "Validation")
print_metrics(metrics_test, "Test")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.linear_model import Ridge

# Load and prepare data
df = pd.read_csv('/content/drive/MyDrive/CPCB.csv', parse_dates=['Datetime'])
df.set_index('Datetime', inplace=True)

features = ['PM2.5', 'SO2', 'CO', 'PM10', 'Temp', 'RH']
df['CO'] = df['CO'].rolling(window=3).mean()
df['_roll3'] = df['CO'].rolling(90).mean()
df['Temp_roll3'] = df['Temp'].rolling(90).mean()
df = df.dropna()

df = df[features].dropna()

# Split data
n = len(df)
train_end = int(n * 0.7)
val_end = int(n * 0.85)

train = df.iloc[:train_end]
val = df.iloc[train_end:val_end]
test = df.iloc[val_end:]

# Lag function
def create_lagged_data(df, lags=3):
    df_lagged = df.copy()
    for col in df.columns:
        for lag in range(1, lags + 1):
            df_lagged[f'{col}_lag{lag}'] = df[col].shift(lag)
    return df_lagged.dropna()

# TAR model with polynomial features
def fit_tar_model_poly(df_lagged, threshold):
    poly = PolynomialFeatures(degree=1, include_bias=False)

    regime_low = df_lagged[df_lagged['CO_lag1'] <= threshold]
    regime_high = df_lagged[df_lagged['CO_lag1'] > threshold]

    X_low = regime_low.drop(columns='CO')
    y_low = regime_low['CO']
    X_high = regime_high.drop(columns='CO')
    y_high = regime_high['CO']

    X_low_poly = poly.fit_transform(X_low)
    X_high_poly = poly.fit_transform(X_high)

    model_low = Ridge(alpha=1.0).fit(X_low_poly, y_low)  # Was 40.0
    model_high = Ridge(alpha=1.0).fit(X_high_poly, y_high)


    return model_low, model_high, threshold, poly

# Prediction function
def predict_tar_poly(df_lagged, model_low, model_high, threshold, poly):
    X = df_lagged.drop(columns='CO')
    X_poly = poly.transform(X)
    condition = df_lagged['CO_lag1'] <= threshold
    preds = np.where(condition,
                     model_low.predict(X_poly),
                     model_high.predict(X_poly))
    return preds

# Grid search over threshold
df_lagged = create_lagged_data(df, lags=3)

train_lagged = df_lagged.loc[train.index.intersection(df_lagged.index)]
val_lagged = df_lagged.loc[val.index.intersection(df_lagged.index)]
test_lagged = df_lagged.loc[test.index.intersection(df_lagged.index)]

# Manually set best threshold and fit model
best_thresh = 150.06
model_low, model_high, _, poly = fit_tar_model_poly(train_lagged, best_thresh)
best_models = (model_low, model_high, poly)

# Optional: evaluate val R² again
yhat_val = predict_tar_poly(val_lagged, model_low, model_high, best_thresh, poly)
best_r2 = r2_score(val_lagged['CO'], yhat_val)
print(f"\n Best threshold used: {best_thresh:.2f} with validation R²: {best_r2:.4f}")


# Final model
model_low, model_high, poly = best_models

# Final predictions
y_train = train_lagged['CO']
y_val = val_lagged['CO']
y_test = test_lagged['CO']

yhat_train = predict_tar_poly(train_lagged, model_low, model_high, best_thresh, poly)
yhat_val = predict_tar_poly(val_lagged, model_low, model_high, best_thresh, poly)
yhat_test = predict_tar_poly(test_lagged, model_low, model_high, best_thresh, poly)

# Metrics
def mase(y_true, y_pred, naive_pred):
    return np.mean(np.abs(y_true - y_pred)) / np.mean(np.abs(y_true[1:] - naive_pred[:-1]))

def evaluate(y_true, y_pred):
    naive = y_true.shift(1).dropna()
    y_true = y_true.iloc[1:]
    y_pred = y_pred[1:]
    return {
        'R2': r2_score(y_true, y_pred),
        'MAE': mean_absolute_error(y_true, y_pred),
        'RMSE': np.sqrt(mean_squared_error(y_true, y_pred)),
        'MASE': mase(y_true, y_pred, naive)
    }

print("\n Final Evaluation with Polynomial TAR:")
print("Train:", evaluate(y_train, yhat_train))
print("Val:  ", evaluate(y_val, yhat_val))
print("Test: ", evaluate(y_test, yhat_test))

# Plots
def plot_actual_vs_pred(y_true, y_pred, title):
    plt.figure(figsize=(10, 4))
    plt.plot(y_true.index, y_true, label='Actual')
    plt.plot(y_true.index, y_pred, label='Predicted')
    plt.title(title)
    plt.xlabel('Time')
    plt.ylabel('CO')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

plot_actual_vs_pred(y_train, yhat_train, 'Train: Actual vs Predicted')
plot_actual_vs_pred(y_val, yhat_val, 'Validation: Actual vs Predicted')
plot_actual_vs_pred(y_test, yhat_test, 'Test: Actual vs Predicted')

# Residuals ACF and PACF
residuals = y_test - yhat_test
plt.figure(figsize=(10, 4))
plot_acf(residuals, lags=30)
plt.title("ACF of Test Residuals")
plt.show()

plt.figure(figsize=(10, 4))
plot_pacf(residuals, lags=30)
plt.title("PACF of Test Residuals")
plt.show()

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import ParameterGrid
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

# Create lagged features for residual correction (train only on past data)
def create_residual_features(df_lagged, residuals, lags=3):
    df_feat = df_lagged.copy()
    df_feat['residuals'] = residuals
    for lag in range(1, lags + 1):
        df_feat[f'residuals_lag{lag}'] = df_feat['residuals'].shift(lag)
    df_feat = df_feat.dropna()
    return df_feat

# Step 1: Compute residuals for train/val/test
residuals_train = y_train - yhat_train
residuals_val = y_val - yhat_val
residuals_test = y_test - yhat_test

# Step 2: Add residuals to original lagged features
train_feat = create_residual_features(train_lagged.copy(), residuals_train, lags=3)
val_feat = create_residual_features(val_lagged.copy(), residuals_val, lags=3)
test_feat = create_residual_features(test_lagged.copy(), residuals_test, lags=3)

# Ensure alignment
res_target_train = train_feat['residuals']
res_target_val = val_feat['residuals']
res_target_test = test_feat['residuals']

# Drop target from features
train_feat_X = train_feat.drop(columns=['residuals'])
val_feat_X = val_feat.drop(columns=['residuals'])
test_feat_X = test_feat.drop(columns=['residuals'])
# max_depth': 7, 'max_features': 'sqrt', 'min_samples_leaf': 5, 'n_estimators': 100

# Step 3: Fit Random Forest on training residuals
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(train_feat_X, res_target_train)

# Step 4: Predict residuals and add to TAR prediction
res_pred_train = rf.predict(train_feat_X)
res_pred_val = rf.predict(val_feat_X)
res_pred_test = rf.predict(test_feat_X)

# Align predicted residuals with TAR predictions
yhat_train_corr = yhat_train[-len(res_pred_train):] + res_pred_train
yhat_val_corr = yhat_val[-len(res_pred_val):] + res_pred_val
yhat_test_corr = yhat_test[-len(res_pred_test):] + res_pred_test

# True values (aligned)
y_train_corr = y_train[-len(res_pred_train):]
y_val_corr = y_val[-len(res_pred_val):]
y_test_corr = y_test[-len(res_pred_test):]

# Step 5: Evaluate corrected predictions
print("\n Final Evaluation after Residual Correction with RF:")
print("Train:", evaluate(y_train_corr, yhat_train_corr))
print("Val:  ", evaluate(y_val_corr, yhat_val_corr))
print("Test: ", evaluate(y_test_corr, yhat_test_corr))

# Plot corrected vs actual
plot_actual_vs_pred(y_test_corr, yhat_test_corr, "Test: Actual vs Corrected TAR+RF Forecast")

# ACF/PACF of corrected residuals
final_residuals = y_test_corr - yhat_test_corr
plt.figure(figsize=(10, 4))
plot_acf(final_residuals, lags=30)
plt.title("ACF of Final Residuals (TAR + RF)")
plt.show()

plt.figure(figsize=(10, 4))
plot_pacf(final_residuals, lags=30)
plt.title("PACF of Final Residuals (TAR + RF)")
plt.show()

# Final Evaluation Metrics for Corrected Forecast

print("\n Final Evaluation Metrics (TAR + RF Residual Correction):")
metrics_train = evaluate(y_train_corr, yhat_train_corr)
metrics_val = evaluate(y_val_corr, yhat_val_corr)
metrics_test = evaluate(y_test_corr, yhat_test_corr)

# Print all metrics cleanly
def print_metrics(metrics, label):
    print(f"\n {label} Set Metrics:")
    for k, v in metrics.items():
        print(f"{k}: {v:.4f}")

print_metrics(metrics_train, "Train")
print_metrics(metrics_val, "Validation")
print_metrics(metrics_test, "Test")
